In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import joblib
import numpy

In [ ]:
# Load the course ratings dataset
ratings_data = pd.read_csv('employee_course_ratings.csv')

# Extract relevant columns
course_ratings = ratings_data[['CourseID', 'CourseName', 'Rating']]

In [ ]:
def create_course_average_ratings(course_ratings):
    # Calculate the average rating for each course
    average_ratings = course_ratings.groupby('CourseName')['Rating'].mean().reset_index()
    # Remove duplicate course names to get unique courses
    unique_courses = average_ratings.drop_duplicates(subset='CourseName')
    # Rename the columns to 'Courses' and 'AverageRating'
    unique_courses = unique_courses.rename(columns={'CourseName': 'Courses', 'Rating': 'AverageRating'})
    return unique_courses

# Create the DataFrame
course_average_ratings = create_course_average_ratings(course_ratings)


print(course_average_ratings)


In [ ]:
def recommend_courses_based_on_score(predicted_score, course_average_ratings):
    # Define a mapping of predicted scores to rating groups
    rating_group_mapping = {
        10: 5,
        9: 5,
        8: 4,
        7: 4,
        6: 3,
        5: 3,
        4: 2,
        3: 2,
        2: 1,
        1: 1,
    }

    # Ensure the predicted score is within the range of available rating groups
    predicted_score = int(round(predicted_score))
    if predicted_score < 1:
        predicted_score = 1
    elif predicted_score > len(rating_group_mapping):
        predicted_score = len(rating_group_mapping)

    # Map the predicted score to the corresponding rating group
    predicted_rating_group = rating_group_mapping.get(predicted_score, 0)  # Default to 0 if score not found

    
    # Filter courses based on the specified rating group
    courses_in_rating_group = course_average_ratings[course_average_ratings['AverageRating'] == predicted_rating_group]

    # Get the unique course names in the rating group
    recommended_courses = list(set(courses_in_rating_group['Courses']))
    
    return recommended_courses

In [ ]:
# Test the Model:

# Load the trained model
trained_model = joblib.load('Employee_Perfomance_Prediction.joblib')

# Define the feature names
feature_names = ['YearsExperience', 'TechnicalSkills', 'CommunicationSkills', 'ProblemSolving', 'Leadership']

# Input the new data with feature names
new_data = pd.DataFrame([[1.0, 2.5, 1.8, 2.2, 1.3]], columns=feature_names)

# Use the trained model to make predictions
predicted_scores = trained_model.predict(new_data)[0]

predicted_employee_score = predicted_scores  # Extract the first element of the predicted_scores array
recommended_courses = recommend_courses_based_on_score(predicted_scores, course_average_ratings)
print(f"Recommended courses for predicted score {predicted_scores}:")
for i, course in enumerate(recommended_courses):
    print(f"{i+1}. {course}")